In [1]:
# essential libraries
import os
from datetime import datetime as dt

# storing and anaysis
import numpy as np
import pandas as pd

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# visualization
# matplot and sns 나중에 안쓰면 지우기
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from PIL import Image

# converter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

#한글깨짐 방지(0420)
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

#pandas와 같이 plotly에서도 간단한 시각화 지원
import cufflinks as cf
cf.go_offline(connected=True)

# for offline ploting
# ===================
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['rgb(31, 119, 180)', 'rgb(255, 127, 14)',
                       'rgb(44, 160, 44)', 'rgb(214, 39, 40)',
                       'rgb(148, 103, 189)', 'rgb(140, 86, 75)',
                       'rgb(227, 119, 194)', 'rgb(127, 127, 127)',
                       'rgb(188, 189, 34)', 'rgb(23, 190, 207)']

layout_setting = {'xaxis_title':'Date','yaxis_title':'Number','font':dict(size=15,color='#60606e',family='Franklin Gothic' )}

# Topic 3. 왜 수도권은 다른 지역과 다른 양상을 보이는 가?

In [3]:
timeProvince = pd.read_csv('TimeProvince.csv',)

FileNotFoundError: [Errno 2] File b'TimeProvince.csv' does not exist: b'TimeProvince.csv'

In [ ]:
youngnam = ['Busan','Daegu','Gyeongsangbuk-do','Gyeongsangnam-do','Ulsan']
honam = ['Gwangju','Jeollabuk-do','Jeollanam-do']
hosae = ['Chungcheongbuk-do','Chungcheongnam-do','Daejeon','Sejong']
sudo = ['Gyeonggi-do','Incheon','Seoul']
gandong = ['Gangwon-do']

In [ ]:
confirmed = pd.pivot_table(timeProvince, index='date',columns='province', values='confirmed')

sub_title = ['수도권 지역','영남 지역','호서 지역','호남 지역','관동 지역']
fig = make_subplots(rows=5,cols=1,
                    subplot_titles=([f'<b>{prov}</b>' for prov in sub_title]),
                    specs=[[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}],[{"secondary_y": True}]])

provinces = [sudo, youngnam, honam, hosae, gandong]
for i in range(5):
    legend=False
    if i==4:
        legend=True
        
    # 지역 추출
    df = pd.DataFrame(confirmed.loc[:,provinces[i]].sum(axis=1))
    
    # 그래프 만들기
    fig.add_trace(go.Bar(name='신규 확진자', x=df.index, y=df.diff().fillna(0).sum(axis=1),
                         marker=dict(color = DEFAULT_PLOTLY_COLORS[3]), showlegend=legend),
                  secondary_y=False, row=i+1, col=1)
    fig.add_trace(go.Scatter(name = '누적 확진자',x=df.index, y= df.sum(axis=1),
                             marker=dict(color = DEFAULT_PLOTLY_COLORS[9]), showlegend=legend),
                 secondary_y=True, row=i+1, col=1)

fig.update_layout(title ='<b>지역별 신규 확진자와 누적 확진자 수<b>',
                  width=1000, height=1500,
                  legend=dict(x=1., y=0.00, traceorder="normal",font=dict(family="sans-serif", size=14)),
                  **layout_setting)
fig.show()

수도권, 영남, 호서, 호남, 관동 지역에 대해서 신규 확진자와 누적 확진자 그래프를 그려 보았을때 수도권 지역 외 다른 지역에서는 코로나 확진자가 급증하기 시작한 31번 확진자 발생일은 2월 17일 이후인 2월 말에서 3월 초에 확진자 수가 크게 증가하였습니다.하지만 수도권 지역에서는 2월말과 3월 초보다는 3월 말부터 많아지기 시작하였습니다. 왜 수도권 지역은 다른 지역과 다르게 증가를 하였는지 알아보도록 하겠습니다.

이후 분석에서는 감염자가 수가 많은 수도권과 영남지역을 중심으로 하겠습니다.

## 1. 2월 말 감염자 수가 급증한 이유는?

In [ ]:
patientInfo = pd.read_csv('PatientInfo.csv')

# Nan으로 되어 있는 것 None으로 변경
patientInfo = patientInfo.fillna('None')

# 케이스 카테고리화
church=['Onchun Church','Dongan Church','River of Grace Community Church','Geochang Church','Pilgrimage to Israel']
nurshing=['Gyeongsan Seorin Nursing Home',"Eunpyeong St. Mary's Hospital",'Bonghwa Pureun Nursing Home','Milal Shelter','Suyeong-gu Kindergarten','Gyeongsan Jeil Silver Town','Cheongdo Daenam Hospital','Gyeongsan Cham Joeun Community Center']
gym=['gym facility in Sejong','gym facility in Cheonan']
etc=['Changnyeong Coin Karaoke','Seongdong-gu APT', 'etc']
work = ['Guro-gu Call Center','Ministry of Oceans and Fisheries']
shincheonji = ['Shincheonji Church', 'None']

category_name = ['Church related','Nurshing related', 'Gym related', 'ETC', 'Workplace related', 'Shincheonji Church']
for i,category in enumerate([church, nurshing, gym, etc, work, shincheonji]):
    patientInfo.infection_case = patientInfo.infection_case.replace(category, category_name[i])
    
# 지역별로 분류해서 그래프 만들기
sub_title=["수도권 지역", "영남 지역","호남 지역","호서 지역","관동 지역"]
fig = make_subplots(rows=2, cols=3,
                    specs=[[{'type':'domain'},{'type':'domain'}, {'type':'domain'}],[{'type':'domain'},{'type':'domain'}, {'type':'domain'}]],
                   subplot_titles=([f'<b>{prov}</b>' for prov in sub_title]))

province_name = [sudo, youngnam, honam, hosae, gandong]
for i, province in enumerate(province_name):
    if i//3 ==0:
        row = 1
        col = (i%3)+1
    else:
        row=2
        col = ((i-3)%3)+1

    df = pd.DataFrame()
    for city_name in province:
        sub = patientInfo[(patientInfo.confirmed_date >='2020-02-25')&(patientInfo.confirmed_date <='2020-03-09')&
                          (patientInfo.province==city_name)]
        df = pd.concat([df,sub],ignore_index=True)
    df = pd.pivot_table(df, index='confirmed_date', columns='infection_case',values='patient_id',aggfunc='count').ffill().fillna(0).cumsum()
    fig.add_trace(go.Pie(labels=list(df.columns), values=df.iloc[-1,:].values.tolist()), row=row, col=col)

fig.update_traces(textinfo="percent")
fig.update_layout(title='<b>2월20일부터 3월9일까지의 감염경로</b>',
                  font = layout_setting['font'],
                  width = 1000, height=700,
                  legend=dict(x=0.75, y=0.0, traceorder="normal",
                             font=dict(family="sans-serif", size=15))) 

감염자의 접촉으로 인한 감염을 제외한다면 수도권 지역외 대부분의 지역이 신천지로 인한 감염자의 비율이 높았습니다. 이는 31번 감염자로 인한 신천지 집단 감염이 많이 발생하였습니다. 때문에 신천지 관련 확진자가 많았던 수도권 제외한 다른 지역에서 확진자가 급격히 증가하였다고 볼 수 있습니다. 신천지관련 감염자가 낮은 비율을 차지 하고 있던 수도권 지역은 집단 감염 발생이 낮아 다른 지역과 다르게 2월말부터 3월초에 신규 확진자 수가 급증하지 않았다고 생각합니다.

**결론**

2월 말부터 3월 초의 신규 확진자의 급증의 원인은 신천지관련 확진자로 신천지 관련 확진자가 적었던 수도권에서는 다른 지역과 다르게 같은 기간에 급증하는 모습을 보이지 않았습니다.

## 2. 확진자의 급증 후 급감하는 다른 지역과 다르게 급감하지 않는 수도권 그 이유는?

### 2.1) 확진자의 감염 경로 추이

In [ ]:
# 날짜 맞추기 => x 축 맞추기
latest_date = []
oldest_date=[]
province_name = [sudo, youngnam]
for province in province_name:
    dates=[]
    for city in province:
        dates += list(set(patientInfo[patientInfo.province==city].confirmed_date))
    latest_date.append(max(dates))
    oldest_date.append(min(dates))
period = [max(oldest_date),min(latest_date)]

# columns(항목 8개)
all_columns = ['contact with patient', 'ETC', 'Nurshing related', 'Workplace related',
               'overseas inflow', 'Shincheonji Church', 'Church related', 'Gym related']


sub_title=["수도권 지역", "영남 지역"]
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=([f'<b>{prov}</b>' for prov in sub_title]))

province_name = [sudo, youngnam]
for i, province in enumerate(province_name):
    row = i+1
    col = 1
    legend=False
    if i==1:
        legend=True
        
    # 지역분류
    df = pd.DataFrame()
    for city_name in province:
        sub = patientInfo[(patientInfo.confirmed_date >=period[0])&(patientInfo.confirmed_date <=period[1])&
                          (patientInfo.province==city_name)]
        df = pd.concat([df,sub],ignore_index=True)
    df = pd.pivot_table(df, index='confirmed_date', columns='infection_case',values='patient_id',aggfunc='count').ffill().fillna(0).cumsum()
    
    # 없는 항목 0으로 채우기
    need_column = set(all_columns) - set(df.columns)
    for name in need_column:
        df[name] = 0
        
    # 그래프
    for i,name in enumerate(all_columns):
        fig.add_trace(go.Scatter(x=df.index, y=df[name],
                                 marker=dict(color = DEFAULT_PLOTLY_COLORS[i]),
                                 mode='lines', name=name, showlegend=legend), row=row, col=col)   


fig.update_layout(title='<b>수도권과 영남지역의 누적 확진자 감염 경로</b>',
                  font = layout_setting['font'],
                  width = 1000, height=700,
                  legend=dict(x=1., y=0.0, traceorder="normal",
                             font=dict(family="sans-serif", size=15)))

fig.show()

In [ ]:
case = pd.read_csv('Case.csv')
dfs = []
for province in [sudo, youngnam]:
    df = pd.DataFrame()
    for city in province:
        df = pd.concat([df, case[case.province==city]])
    df = pd.pivot_table(df, columns='infection_case', values='confirmed', aggfunc='sum')
    dfs.append(df)
    
provinces=["수도권 지역", "영남 지역"]
fig = make_subplots(rows=1, cols=2,
                    horizontal_spacing=0.03,
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                   subplot_titles=([f'<b>{prov}</b>' for prov in provinces]))

for i,df in enumerate(dfs):
    df = df.sort_values(by='confirmed', axis=1, ascending=False)
    etc = df.iloc[:,6:].sum(axis=1).values[0]
    df = df.iloc[:,:6]
    df['etc'] = etc
    fig.add_trace(go.Pie(labels=list(df.columns), values=df.values[0].tolist()), row=1, col=i+1)

fig.update_layout(title='<b>수도권과 영남지역의 확진자 감염 경로 비율</b>',
                  font = layout_setting['font'],
                  width = 1000, height=400,
                  legend=dict(x=1., y=0.0, traceorder="normal",
                             font=dict(family="sans-serif", size=15)))
fig.show()

수도권 지역은 감염자 접촉과 해외유입으로 인한 감염자가 많았고 영남지역은 기타 감염 경로로인한 감염자가 많다는 것을 볼 수 있었습니다. 질병관리 본부에서 발표한 자료에서 기타로 분류된 대부분을 신천지 관련으로 볼 수 있다고 한 것과 Case자료를 참고하였을때, 기타의 대부분은 신천지 관련으로 볼 수 있다는 것을 알 수 있습니다. 영남지역의 기타를 신천지 관련이 가장 높은 감염 경로라고 볼 수 있습니다.
수도권 지역에서 해외 유입으로 인한 감염이 다른 감염 경로과 다르게 뒤늦게 증가하는 것을 볼 수 있었습니다.  

## 글 고쳤어요

**그래프 그리기 전**

질병관리 본부에서 발표한 자료에서 대구와 경북의 감염경로를 알 수 없는 것의 대부분을 신천지 관련으로 볼 수 있다고 한 것과 Case자료를 참고하여때, 감염경로가 Nan을 신천지로 인한 감염으로 판단하고 그래프를 그렸습니다.

**그래프 그리고 난 후**

수도권 지역은 감염자 접촉과 해외유입으로 인한 감염자가 많았고 영남지역은 신천지로인한 감염자가 많다는 것을 볼 수 있었습니다.
수도권 지역에서 해외 유입으로 인한 감염이 다른 감염 경로과 다르게 뒤늦게 증가하는 것을 볼 수 있었습니다.  

### 1.2) 해외 유입 감염자가 신규 감염자에 있어서 큰 영향을 미칠까?
2020.04.14부터 2020.04.27동안 질병관리본부에서 발표한 보도자료에서 추출하여 감염경로를 살펴 보았습니다.

In [ ]:
# 지역 이름이 다르게 설정되어있어서 다시 설정
youngnam = ['Busan','Daegu','Gyungbuk','Gyungnam','Ulsan']
honam = ['Gwangju','Jeonbuk','Jeonnam']
hosae = ['Chungbuk','Chungnam','Daejeon','Sejong']
sudo = ['Gyungki','Incheon','Seoul']
gandong = ['Gangwon']

df = pd.DataFrame()
for date in range(20200414, 20200428):
    sub = pd.read_excel('PatientTrend_20200414_20200427.xlsx', sheet_name=str(date))
    date = str(date)
    sub['Date'] = str(date[:4])+'-'+str(date[4:6])+'-'+str(date[6:])
    df = pd.concat([df,sub])
    
df['Overseas Inflow'] = df.Overseas + df.Oversea_inflow_related
df.columns = ['City', 'Sum', 'Overseas', 'Shincheonji Church', 'Mass Infection','Contact with Patient', 'Oversea_inflow_related', 'ETC', 'Date','Overseas Inflow']
df = df[['Date', 'City', 'Overseas Inflow', 'Shincheonji Church', 'Mass Infection', 'Contact with Patient', 'ETC', 'Sum']]

total = []
for province in [sudo, youngnam]:
    sub = pd.DataFrame()
    a = pd.DataFrame()
    for city in province:
        sub = df[df.City==city]
        a[city] = (sub.iloc[-1,2:] - sub.iloc[0,2:])
    a = np.where(a<0,0,a).sum(axis=1)
    total.append(a)

total = pd.DataFrame(total, index=['수도권', '영남'], columns=['Overseas Inflow', 'Shincheonji Church', 'Mass Infection', 'Contact with Patient', 'ETC', 'Sum'])
total

In [ ]:
provinces=["수도권 지역", "영남 지역"]
fig = make_subplots(rows=1, cols=2,
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                   subplot_titles=([f'<b>{prov}</b>' for prov in provinces]))

index=list(total.columns[:-1])
fig.add_trace(go.Pie(labels=index, values=total.iloc[0,:].tolist()), row=1, col=1)
fig.add_trace(go.Pie(labels=index, values=total.iloc[1,:].tolist()), row=1, col=2)

fig.update_traces(textinfo="percent+label")
fig.update_layout(title='<b>4월14일부터 4월27일동안의 감염 경로</b>',
                  font = layout_setting['font'],
                  width = 1100, height=400,
                  legend=dict(x=1., y=0.0, traceorder="normal",
                             font=dict(family="sans-serif", size=15))) 

##### 결론
수도권 지역의 해외 유입 감염자 수가 뒤늦게 증가하여 최근 이슈화가 되고 있지만 최근 2주동안 신규 확진자의 총 수를 보았을 때, 수도권 61명, 영남권 64명으로 차이가 나지 않는 것을 알 수 있습니다.그렇기때문에 해외 유입으로 인한 감염이 신규 확진자 증가에 영향이 있다고 보기에 어렵다는 것을 알 수 있었습니다.